<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/GPT_finetunning_aviation_validationdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://platform.openai.com/docs/guides/fine-tuning

In [ ]:
!pip install colab-env --upgrade -q
!pip install openai -q

In [ ]:
import colab_env
import os

from openai import OpenAI
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [3]:
from openai import OpenAI
client = OpenAI()

modellist=client.models.list()
modellist.data

[Model(id='gpt-3.5-turbo-0613', created=1686587434, object='model', owned_by='openai'),
 Model(id='whisper-1', created=1677532384, object='model', owned_by='openai-internal'),
 Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-16k', created=1683758102, object='model', owned_by='openai-internal'),
 Model(id='tts-1-hd-1106', created=1699053533, object='model', owned_by='system'),
 Model(id='tts-1-hd', created=1699046015, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo', created=1677610602, object='model', owned_by='openai'),
 Model(id='gpt-4-turbo-2024-04-09', created=1712601677, object='model', owned_by='system'),
 Model(id='gpt-4-0125-preview', created=1706037612, object='model', owned_by='system'),
 Model(id='gpt-4-turbo-preview', created=1706037777, object='model', owned_by='system'),
 Model(id='gpt-4-turbo', created=1712361441, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-instruct-0914', created=1694122

In [4]:
modellist.data[33]

Model(id='gpt-3.5-turbo-0301', created=1677649963, object='model', owned_by='openai')

In [5]:
#for i in range(34,len(modellist.data)):
#  print(modellist.data[i].id)
#  client.models.delete(modellist.data[i].id)

In [7]:
len(modellist.data) #34 Official by OpenAI

34

Fine-tuning is currently available for the following models: gpt-3.5-turbo-0125 (recommended), gpt-3.5-turbo-1106, gpt-3.5-turbo-0613, babbage-002, davinci-002, and gpt-4-0613 (experimental).

In [8]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  messages=[
    {
      "role": "system",
      "content": "Summarize content you are provided with for a second-grade student."
    },
    {
      "role": "user",
      "content": "Jupiter is the fifth planet from the Sun and the largest in the Solar System. It is a gas giant with a mass one-thousandth that of the Sun, but two-and-a-half times that of all the other planets in the Solar System combined. Jupiter is one of the brightest objects visible to the naked eye in the night sky, and has been known to ancient civilizations since before recorded history. It is named after the Roman god Jupiter.[19] When viewed from Earth, Jupiter can be bright enough for its reflected light to cast visible shadows,[20] and is on average the third-brightest natural object in the night sky after the Moon and Venus."
    }
  ],
  temperature=0.7,
  max_tokens=64,
  top_p=1
)

In [9]:
print(response.choices[0].message.content)

Jupiter is a big planet that is fifth from the Sun and is the largest in our Solar System. It is made mostly of gas and is very bright in the night sky. People have known about Jupiter for a long time and it is named after a Roman god. When we look at Jupiter from Earth, it can


AVIATION-QA

In [11]:
import json

def get_data(path):
    #read each file and retrieve the contexts, qustions and answers
    data = []
    with open(path, 'r') as f:  # Open in text mode for JSON Lines
        for line in f:
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Error decoding line: {line}")
                print(e)
    return data  # Return the list of parsed JSON objects

In [12]:
# Upload the training file FOR SQuAD v2 1000 QUESTIONS
#filename_path='/content/gdrive/MyDrive/datasets/dataft-L1000.jsonl'

# aviation TRAIN QUESTION
filename_path='/content/gdrive/MyDrive/datasets/train_aviation-gpt.jsonl'
data=get_data(filename_path)
#data[10]

# aviation TEST QUESTION
test_filename_path='/content/gdrive/MyDrive/datasets/test_aviation-gpt.jsonl'
data_test=get_data(test_filename_path)

In [13]:
numberofquestions=len(data)
numberofquestions

15000

In [14]:
numberofquestions_test=len(data_test)
numberofquestions_test

10807

In [15]:
#nrec=8
nrec=2
data[nrec]['messages'][0]['content']

question=data[nrec]['messages'][1]['content']
answer=data[nrec]['messages'][2]['content']

print('Record #: %s'%nrec)
print('Question: %s'%question)
print('Answer by the dataset: %s'%answer)

Record #: 2
Question: Who determines the probable cause(s) of this accident no. ERA22LA104?
Answer by the dataset: The National Transportation Safety Board


In [16]:
nrec=2
data_test[nrec]['messages'][0]['content']

question=data_test[nrec]['messages'][1]['content']
answer=data_test[nrec]['messages'][2]['content']

print('Record #: %s'%nrec)
print('Question: %s'%question)
print('Answer by the dataset: %s'%answer)

Record #: 2
Question: What was the flight conducted under for the accident no. GAA18CA423?
Answer by the dataset: Part 91: General aviation - Personal


In [17]:
from openai import OpenAI
client = OpenAI()

# Upload the training file
train_file_response = client.files.create(
  file=open(filename_path, "rb"),
  purpose="fine-tune"
)


# Upload the training file
test_file_response = client.files.create(
  file=open(test_filename_path, "rb"),
  purpose="fine-tune"
)


# Get the file ID
tain_file_id = train_file_response.id
test_file_id = test_file_response.id

# Start the fine-tuning job using the file ID
cftjc=client.fine_tuning.jobs.create(
  training_file=tain_file_id,
  validation_file=test_file_id,
  model="gpt-3.5-turbo-0125"
)

training_file=cftjc.training_file
fine_tuning_job=cftjc.id

In [18]:
# List 10 fine-tuning jobs
#client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
#client.fine_tuning.jobs.retrieve(fine_tuning_job)

# Cancel a job
#client.fine_tuning.jobs.cancel("ftjob-HHsRv4STcKEaAywwrYDjL4zI")

# List up to 10 events from a fine-tuning job
#client.fine_tuning.jobs.list_events(fine_tuning_job_id=fine_tuning_job, limit=10)

# Delete a fine-tuned model (must be an owner of the org the model was created in)
#client.models.delete("ft:gpt-3.5-turbo:acemeco:suffix:abc123")

In [19]:
actual_job_id = fine_tuning_job  # Replace with a real ID
fine_tuning_job_retrieve=client.fine_tuning.jobs.retrieve(actual_job_id)
client.fine_tuning.jobs.retrieve(actual_job_id)

FineTuningJob(id='ftjob-mcFf0F6JsG4URKmmc5RspojZ', created_at=1718948034, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-kw9OnWtmHIJ748Xe7oTALKxl', result_files=[], seed=886489863, status='validating_files', trained_tokens=None, training_file='file-ZFpygfKNqYA4C1Vdigg2tBJI', validation_file='file-TgBiuZsGINqTPYCXceIGfihf', estimated_finish=None, integrations=[], user_provided_suffix=None)

In [20]:
organization_id=fine_tuning_job_retrieve.organization_id
fine_tuned_model=fine_tuning_job_retrieve.fine_tuned_model
print(fine_tuned_model)

None


https://platform.openai.com/finetune/

In [21]:
while fine_tuned_model == None:
       fine_tuning_job_retrieve=client.fine_tuning.jobs.retrieve(actual_job_id)
       organization_id=fine_tuning_job_retrieve.organization_id
       fine_tuned_model=fine_tuning_job_retrieve.fine_tuned_model


In [22]:
print('Fine Tuned Model: %s'%fine_tuned_model)

Fine Tuned Model: ft:gpt-3.5-turbo-0125:xamrysoft::9cSMOUWb


In [23]:
from openai import OpenAI
client = OpenAI()


completion = client.chat.completions.create(
  model=fine_tuned_model,
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": question}
  ]
)
print('PROMPT: %s'%question)
print('ANSWER GENERATED: %s'%completion.choices[0].message.content)

PROMPT: What was the flight conducted under for the accident no. GAA18CA423?
ANSWER GENERATED: Part 91: General aviation - Personal


In [24]:
modellist=client.models.list()
modellist.data

[Model(id='gpt-3.5-turbo-0613', created=1686587434, object='model', owned_by='openai'),
 Model(id='whisper-1', created=1677532384, object='model', owned_by='openai-internal'),
 Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-16k', created=1683758102, object='model', owned_by='openai-internal'),
 Model(id='tts-1-hd-1106', created=1699053533, object='model', owned_by='system'),
 Model(id='tts-1-hd', created=1699046015, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo', created=1677610602, object='model', owned_by='openai'),
 Model(id='gpt-4-turbo-2024-04-09', created=1712601677, object='model', owned_by='system'),
 Model(id='gpt-4-0125-preview', created=1706037612, object='model', owned_by='system'),
 Model(id='gpt-4-turbo-preview', created=1706037777, object='model', owned_by='system'),
 Model(id='gpt-4-turbo', created=1712361441, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-instruct-0914', created=1694122

In [27]:
modellist.data[34]

Model(id='ft:gpt-3.5-turbo-0125:xamrysoft::9cSMOUWb', created=1718952508, object='model', owned_by='user-mgtzrbiiso3g68oicniq0obp')

In [28]:
actual_job_id = fine_tuning_job  # Replace with a real ID
fine_tuning_job_retrieve=client.fine_tuning.jobs.retrieve(actual_job_id)
client.fine_tuning.jobs.retrieve(actual_job_id)

FineTuningJob(id='ftjob-mcFf0F6JsG4URKmmc5RspojZ', created_at=1718948034, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:xamrysoft::9cSMOUWb', finished_at=1718952507, hyperparameters=Hyperparameters(n_epochs=1, batch_size=10, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-kw9OnWtmHIJ748Xe7oTALKxl', result_files=['file-wkCMmZmIwr2tqVVAtSnGRNTl'], seed=886489863, status='succeeded', trained_tokens=1300090, training_file='file-ZFpygfKNqYA4C1Vdigg2tBJI', validation_file='file-TgBiuZsGINqTPYCXceIGfihf', estimated_finish=None, integrations=[], user_provided_suffix=None)

In [26]:
#for i in range(34,len(modellist.data)):
#  print(modellist.data[i].id)
#  client.models.delete(modellist.data[i].id)